In [1]:
%matplotlib notebook

import os
import time
import numpy as np
import matplotlib as mpl
import sys
from importlib import reload

In [2]:
import qcodes as qc
from broadbean.plotting import plotter

In [3]:
from pytopo.qctools.instruments import create_inst, add2station
from pytopo.qctools.dataset2 import select_experiment

from plottr import client
from plottr.qcodes_dataset import QcodesDatasetSubscriber
from plottr.qcodes_dataset import QcodesDatasetSubscriber as LivePlotSubscriber

from pytopo.mplplots.init_nb_plotting import *

qc.config['core']['db_location'] = r"d:/OneDrive/BF2/Data/experiments.db"
qc.dataset.database.initialise_database()

In [4]:
from qcodes import load_by_id
from qcodes import validators
from qcodes.dataset.measurements import Measurement
from qcodes.dataset.database import initialise_database
from qcodes.instrument.base import Instrument
from qcodes.instrument.parameter import Parameter
from qcodes.dataset.plotting import plot_by_id
from qcodes.utils.validators import Numbers

In [5]:
import qcodes as qc
import broadbean as bb

from broadbean.plotting import plotter

In [6]:
from pytopo.qctools import hard_sweep; reload(hard_sweep)
from pytopo.qctools.hard_sweep import HardSweep, HardSweepDetector

In [7]:
ramp = bb.PulseAtoms.ramp

In [8]:
from pytopo.qctools import instruments as instools; reload(instools)
from pytopo.qctools.instruments import create_inst, add2station
from pytopo.qctools.dataset2 import select_experiment
from pytopo.qctools.measurement import MeasurementExt

from pytopo.rf import alazar_detector; reload(alazar_detector)
from pytopo.rf.alazar_detector import AlazarDetector

from plottr import qcodes_dataset; reload(qcodes_dataset)
from plottr.qcodes_dataset import QcodesDatasetSubscriber

from cqed import broadbean; reload(broadbean)
from cqed.broadbean import BroadBeanSequence, BluePrints

from cqed import bb_sequences; reload(bb_sequences)
from cqed.bb_sequences import TriggeredReadoutSequence



In [9]:
class AWGGateRamp1D(BroadBeanSequence):
    
    trigger_delay = 100e-9
    alazar_trigger_len = 100e-9
    sweep_dims = ['voltages', ]

    
    def __init__(self, *arg, **kw):
        super().__init__(*arg, **kw)
        
        self.add_parameter('initial_delay', set_cmd=None, 
                           unit='s', initial_value=10e-6)
        self.add_parameter('step_len', set_cmd=None, 
                           unit='s', initial_value=10e-6)
        self.add_parameter('start_voltage', set_cmd=None,
                           unit='V', initial_value=0)
        self.add_parameter('stop_voltage', set_cmd=None,
                           unit='V', initial_value=1)
        self.add_parameter('voltage_points', set_cmd=None,
                           unit='', initial_value=11)
        self.add_parameter('ramp_down_time', set_cmd=None,
                           unit='s', initial_value=10e-6)
        
    def setup(self, **kw):
        vpts = np.linspace(self.start_voltage(), self.stop_voltage(), self.voltage_points())
        self.voltages(vpts)
        super().setup(**kw)
        
        
    def sequence(self):
        elem = bb.Element()
        bps = BluePrints(self.chan_map, sample_rate=self.sample_rate())
        
        cut_t = 0
        bps['ro_trigger'] = []
        bps['awg_voltage'].insertSegment(0, ramp, (self.start_voltage(), self.start_voltage()), 
                                         dur=self.initial_delay(), name='voltage')        
        for i, v in enumerate(self.voltages()):
            bps['awg_voltage'].insertSegment(i+1, ramp, (v, v), 
                                             dur=self.step_len(), name='voltage')
            bps['ro_trigger'].append((self.initial_delay() + self.step_len() * i + self.trigger_delay, 
                                      self.alazar_trigger_len))
            
        bps['awg_voltage'].insertSegment(-1, ramp, (v, self.start_voltage()), 
                                         dur=self.ramp_down_time(), name='voltage')        

        for n, bp in bps():
            elem.addBluePrint(n, bp)
            
        seq = bb.Sequence()
        seq.name = 'v_ramp'
        seq.addElement(1, elem)
        
        return seq
        

In [10]:
BroadBeanSequence.chan_map = {
    1 : ['awg_voltage', 'ro_trigger', None],
}
BroadBeanSequence.chan_settings[1]['Vpp'] = 1.0
BroadBeanSequence.chan_settings[1]['marker_hi'] = [1.5, 1.5, 1.5, 1.5]
BroadBeanSequence.chan_settings[1]['marker_lo'] = [-0.3, -0.3, -0.3, -0.3]
BroadBeanSequence.sweep_wait = None

TriggeredReadoutSequence.chan_map = {
    1 : ['FILL.1', 'ro_trigger', None],
}

### Instruments init

In [11]:
sys.path.append('D:/Code')
from RIGOL.RIGOL_driver import DG1062
fg = DG1062(name = 'fg', address = 'USB0::0x1AB1::0x0642::DG1ZA195006391::INSTR')

Connected to: None fg (serial:None, firmware:None) in 5.16s


In [12]:
from qcodes.instrument_drivers.tektronix.AWG5208 import AWG5208
awg = create_inst(AWG5208, "awg", address='TCPIP0::169.254.254.84::inst0::INSTR')
station = qc.Station(awg)

# Alazar
from qcodes.instrument_drivers.AlazarTech import utils; reload(utils)
from qcodes.instrument_drivers.AlazarTech import ATS9360; reload(ATS9360)
from qcodes.instrument_drivers.AlazarTech.ATS9360 import AlazarTech_ATS9360
alazar = create_inst(AlazarTech_ATS9360, 'alazar', force_new_instance=True)
station = add2station(station, alazar)

# Alazar aquisition controllers
from pytopo.rf import alazar_acquisition; reload(alazar_acquisition)
from pytopo.rf.alazar_acquisition import RawAcqCtl, AvgDemodCtl, AvgIQCtl

raw_acq = create_inst(RawAcqCtl, 'raw_acq', 'alazar', force_new_instance=True)
station = add2station(station, raw_acq)

avgdemod_acq = create_inst(AvgDemodCtl, 'avgdemod_acq', 'alazar', force_new_instance=True)
station = add2station(station, avgdemod_acq)

avgiq_acq = create_inst(AvgIQCtl, 'avgiq_acq', 'alazar', force_new_instance=True)
station = add2station(station, avgiq_acq)

# RF sources
from qcodes.instrument_drivers.rohde_schwarz.SGS100A import RohdeSchwarz_SGS100A
LO = create_inst(RohdeSchwarz_SGS100A, 'LO', address="TCPIP0::169.254.2.20", force_new_instance=True)
station = add2station(station, LO)

RF = create_inst(RohdeSchwarz_SGS100A, 'RF', address="TCPIP0::169.254.231.38", force_new_instance=True)
station = add2station(station, RF)

#from qcodes.instrument_drivers.agilent.E8267C import E8267
#S1 = instools.create_inst(E8267, 'S1', address='GPIB0::19::INSTR', force_new_instance=True)
#inst_list.append(S1)

from pytopo.rf.sources import HeterodyneSource
hetsrc = create_inst(HeterodyneSource, 'hetsrc', RF=RF, LO=LO, force_new_instance=True)
station = add2station(station, hetsrc)

awg.wfmxFileFolder = "\\Users\\MSFTE\\Documents"
awg.seqxFileFolder = "\\Users\\MSFTE\\Documents"

Connected to: TEKTRONIX AWG5208 (serial:B020205, firmware:FV:6.0.0242.0) in 0.19s
Connected to: Rohde&Schwarz SGS100A (serial:1416.0505k02/110509, firmware:3.1.19.15-3.50.124.67) in 0.03s
Connected to: Rohde&Schwarz SGS100A (serial:1416.0505k02/110510, firmware:3.1.19.15-3.50.124.67) in 0.04s


In [13]:
def awg_trigger_func(do_start):
    if do_start:
        station.awg.play()
    else:
        station.awg.stop()
        
for ctl in [raw_acq, avgdemod_acq, avgiq_acq]:
    ctl.trigger_func = awg_trigger_func

In [14]:
from qcodes.instrument_drivers.american_magnetics.AMI430 import AMI430,AMI430_3D
ami_x = create_inst(AMI430, "AMI430_x", "169.254.158.77", port = 7180, has_current_rating=True)
ami_y = create_inst(AMI430, "AMI430_y", "169.254.208.179", port = 7180, has_current_rating=True)
ami_z = create_inst(AMI430, "AMI430_z", "169.254.88.255", port = 7180, has_current_rating=True)


ami = create_inst(AMI430_3D, "AMI430", ami_x, ami_y, ami_z, 2)


station.add_component(ami_x)
station.add_component(ami_y)
station.add_component(ami_z)
station.add_component(ami)

Connected to: AMERICAN MAGNETICS INC. 430 (serial:2.01, firmware:None) in 0.87s
Connected to: AMERICAN MAGNETICS INC. 430 (serial:2.01, firmware:None) in 0.89s
Connected to: AMERICAN MAGNETICS INC. 430 (serial:2.01, firmware:None) in 0.80s


'AMI430'

In [60]:
ami.spherical([1, 105, 90])

In [15]:
from qcodes.instrument_drivers.QuTech.IVVI import IVVI
ivvi = create_inst(IVVI, "ivvi", "ASRL4::INSTR")

Initialized IVVI-rack in 0.03s


In [16]:
station.add_component(ivvi)

for i in range(1,16):
    ivvi.parameters['dac{}'.format(i)].set_step(0.5)
    ivvi.parameters['dac{}'.format(i)].set_delay(0.001)
    
ivvi.dac1.set_delay(0.001)
ivvi.dac1.set_step(20)

ivvi.dac2.set_delay(0.001)
ivvi.dac2.set_step(20)

ivvi.dac3.set_delay(0.001)
ivvi.dac3.set_step(2)

ivvi.dac4.set_delay(0.001)
ivvi.dac4.set_step(20)

ivvi.dac5.set_delay(0.001)
ivvi.dac5.set_step(60)

ivvi.dac6.set_delay(0.001)
ivvi.dac6.set_step(2)

ivvi.dac6.set_delay(0.001)
ivvi.dac6.set_step(20)

ivvi.dac7.set_delay(0.001)
ivvi.dac7.set_step(60)

ivvi.dac8.set_delay(0.001)
ivvi.dac8.set_step(20)

ivvi.dac9.set_delay(0.001)
ivvi.dac9.set_step(60)


d:\code\qcodes\qcodes\instrument\parameter.py:577: UserWarning: set_step is deprecated use step property as in `inst.step = stepvalue` instead
  "set_step is deprecated use step property as in `inst.step = "
d:\code\qcodes\qcodes\instrument\parameter.py:589: UserWarning: set_delay is deprecated use inter_delay or post_delay property as in `inst.inter_delay = delayvalue` instead
  "set_delay is deprecated use inter_delay or post_delay property "


In [17]:
class ConversionParameter(Parameter):
    
    def __init__(self, name, src_param, get_conv, set_conv=None, **kw):
        super().__init__(name, **kw)
        self.src_param = src_param
        self.get_conv = get_conv
        self.set_conv = set_conv
    
    def get_raw(self):
        return self.get_conv(self.src_param())
    
    def set_raw(self, value):
        if self.set_conv is None:
            raise NotImplementedError("No set conversion implemented.")
        
        return self.src_param(self.set_conv(value))
    

class IVVISetup(Instrument):
    
    def __init__(self, name, ivvi, **kw):
        super().__init__(name, **kw)
        
        self.ivvi = ivvi
        
        self.add_parameter('voltage_multiplier', get_cmd=None, set_cmd=None, 
                           initial_value=1e-3)
        self.add_parameter('current_multiplier', get_cmd=None, set_cmd=None, 
                           initial_value=1e-7)
        self.add_parameter('iso_in_multiplier', get_cmd=None, set_cmd=None,
                           initial_value=1e-2)
        self.add_parameter('gate_multiplier', get_cmd=None, set_cmd=None, 
                           initial_value=15)
        
        self.add_parameter('M1b_gain', get_cmd=None, set_cmd=None, initial_value=1e6)
        self.add_parameter('M1b_mode', get_cmd=None, set_cmd=None, 
                           vals=validators.Enum('low_noise', 'low_R_in'), 
                           initial_value='low_noise')
        self.add_parameter('M1b_R_in', get_cmd=self._get_M1b_R_in)
        
        self.add_parameter('M2m_Vgain', get_cmd=None, set_cmd=None, initial_value=1e3)

        for i in range(1, self.ivvi._numdacs + 1):       
            self.add_parameter('dac{}'.format(i), unit='V',
                               get_cmd=self.ivvi._gen_ch_get_func(self._get_dac, i),
                               set_cmd=self.ivvi._gen_ch_set_func(self._set_dac, i))
            
    def _get_M1b_R_in(self):
        if self.M1b_mode() == 'low_noise':
            return 2e3 + 1e-3 * self.M1b_gain()
        elif self.M1b_mode() == 'low_R_in':
            return 2e3 + 1e-4 * self.M1b_gain()
            
    def _get_dac(self, ch):
        if ch == 6:
            return self.ivvi.set('dac{}'.format(ch)) * 1e-3 * self.gate_multiplier()
        else:
            return self.ivvi.get('dac{}'.format(ch)) * 1e-3 * self.voltage_multiplier()
    
    def _set_dac(self, ch, val):
        if ch == 3:
            return self.ivvi.set('dac{}'.format(ch), val * 1e3 / self.gate_multiplier())
        else:
            return self.ivvi.set('dac{}'.format(ch), val * 1e3 / self.voltage_multiplier())
    


In [18]:
ivvi_setup = create_inst(IVVISetup, 'ivvi_setup', ivvi=ivvi, force_new_instance=True)
ivvi_setup.voltage_multiplier(10e-3)
#ivvi_setup.voltage_multiplier(100e-9)
ivvi_setup.gate_multiplier(15)

In [61]:
ivvi_setup.dac3(-6.5)
ivvi.dac6(50)
ivvi.dac7(-400)

In [39]:
plungers = np.linspace(-400, 200, 101)

In [ ]:
SAMPLE = 'Frank'

In [19]:
with alazar.syncing():
    alazar.clock_source('INTERNAL_CLOCK')
    alazar.sample_rate(int(0.2e9))
    alazar.clock_edge('CLOCK_EDGE_RISING')
    alazar.external_sample_rate(int(1e9))
    alazar.decimation(1)
    alazar.coupling1('DC')
    alazar.coupling2('DC')
    alazar.channel_range1(0.4)
    alazar.channel_range2(0.4)
    alazar.impedance1(50)
    alazar.impedance2(50)
    alazar.trigger_source1('EXTERNAL')
    alazar.trigger_level1(128 + 20)
    alazar.external_trigger_coupling('DC')
    alazar.external_trigger_range('ETR_2V5')
    alazar.trigger_delay(0)
    alazar.timeout_ticks(int(1e7))

d:\code\qcodes\qcodes\instrument_drivers\AlazarTech\ATS.py:564: UserWarning: Using Internal clock but parameter external_sample_rate is set.This will have no effect and is ignored
  warnings.warn("Using Internal clock but parameter external_sample_rate is set."


### Software sweep

In [18]:
ivvi_setup.dac3(-6.5)
ivvi.dac6(50)
ivvi.dac7(-400)

plungers = np.linspace(-400, -50, 101)

In [63]:
hetsrc.frequency(2.33e8)
hetsrc.IF(50e6)
hetsrc.RF.on()
hetsrc.LO.on()
hetsrc.RF.power(-40)
hetsrc.LO.power(10)

In [86]:
hetsrc.IF(1e6)

In [85]:
hetsrc.RF.power(-40)
hetsrc.LO.power(10)

In [91]:
hetsrc.frequency(2.3e8)

In [90]:
### User settings

rf_frqs = np.linspace(150e6, 250e6, 201)[:] # spec frequencies
plungervals = np.linspace(0, 400, 101)
# ro_atts = np.arange(26, 42, 2)
setup_awg = True
int_time = 0.1e-3
buffer_len = 100e-6


### End User settings


### Setting up instruments

# set IQ acquisition controller
acq = avgiq_acq
acq.trigger_func = None

# create new instance of sequence and detector
sequence = create_inst(TriggeredReadoutSequence, 'sequence', awg=awg, force_new_instance=True)
station = add2station(station, sequence)
detector = create_inst(AlazarDetector, name='detector', acqctl=acq, force_new_instance=True)
station = add2station(station, detector)

# configure the sequence parameters
sequence.pre_trigger_delay(1e-6) # some delay at the beginning
sequence.trigger_len(100e-9) # short trigger for the alazar
sequence.seq_len(10e-6) # quick rep-rate is OK here
if setup_awg:
    sequence.setup(program_awg=True, start_awg=True) # program and start only if something else is running before
else:
    sequence.setup(program_awg=False, start_awg=False)
    
# configure the Alazar detector
detector.set_sweeper(sequence) # set the sequence (it's just one dummy sweep-point here)
detector.demod_frq(hetsrc.IF()) # demod frequency from the sources
detector.acq_time(buffer_len) # not too long integration time ber buffer
detector.configure_alazar(
    records_per_buffer=1,
    buffers_per_acquisition=int(int_time/buffer_len), # set averaging by choosing the number of buffers
)
detector.setup() # this makes sure all params are set correctly

### Set up the measurement info
exp = select_experiment('resonator_spec', 'dev48') # select experiment (info for the database)

# need to make sure the sweep is correctly described in the beginning!
meas = MeasurementExt(station, exp=exp, 
                      hard_sweep_detector=detector,
                      soft_sweep_params=[
                          hetsrc.frequency,
                          ivvi.dac7
#                           ro_attenuator.attenuation,
                      ],
                     )

#meas.register_parameter(ivvi.dac7)
# and finally, the measurement loop
meas.write_period = 2 # how often to write to the DB [s]
with meas.run() as datasaver:
    
    # this is to connect the database to the plottr.
    plot_subscriber = QcodesDatasetSubscriber(datasaver.dataset)
    datasaver.dataset.subscribe(plot_subscriber, state=[])
    
    # the loop itself is trivial
#     for att in ro_atts:
#         ro_attenuator.attenuation(att)
    for p in plungervals:
        ivvi.dac7(p)
        for f in rf_frqs:
            hetsrc.frequency(f)
            time.sleep(0.01)

            detector.acquisition() # this gets the data from the alazar
            datasaver.add_result(*meas.get_result()) # this adds all data to the datasaver.

No ro_gate defined in channel map. No readout gate pulse will be generated.
No src_gate defined in channel map. No src gate pulse will be generated.


Instrument sequence already exists.
Instrument detector already exists.
Waveforms done
['wfm_1_1_1']
True debug
[['wfm_1_1_1']]
Starting experimental run with id: 567


KeyboardInterrupt: 

In [28]:
from qcodes.instrument_drivers.yokogawa.GS200 import GS200
yoko = create_inst(GS200, "yoko", "GPIB0::1::INSTR")

Connected to: YOKOGAWA GS210 (serial:91T926459, firmware:2.02) in 0.06s


In [31]:
yoko.voltage_range(0.01)
yoko.voltage(0)
yoko.current_limit(1e-3)
yoko.on()

In [92]:
biasvals = np.linspace(-400e-6, 400e-6, 101)
plungervals = np.linspace(-1200, 600, 101)

setup_awg = True
int_time = 0.1e-3
buffer_len = 100e-6

hetsrc.frequency(233e6)
### End User settings


### Setting up instruments

# set IQ acquisition controller
acq = avgiq_acq
acq.trigger_func = None

# create new instance of sequence and detector
sequence = create_inst(TriggeredReadoutSequence, 'sequence', awg=awg, force_new_instance=True)
station = add2station(station, sequence)
detector = create_inst(AlazarDetector, name='detector', acqctl=acq, force_new_instance=True)
station = add2station(station, detector)

# configure the sequence parameters
sequence.pre_trigger_delay(1e-6) # some delay at the beginning
sequence.trigger_len(100e-9) # short trigger for the alazar
sequence.seq_len(10e-6) # quick rep-rate is OK here
if setup_awg:
    sequence.setup(program_awg=True, start_awg=True) # program and start only if something else is running before
else:
    sequence.setup(program_awg=False, start_awg=False)
    
# configure the Alazar detector
detector.set_sweeper(sequence) # set the sequence (it's just one dummy sweep-point here)
detector.demod_frq(hetsrc.IF()) # demod frequency from the sources
detector.acq_time(buffer_len) # not too long integration time ber buffer
detector.configure_alazar(
    records_per_buffer=1,
    buffers_per_acquisition=int(int_time/buffer_len), # set averaging by choosing the number of buffers
)
detector.setup() # this makes sure all params are set correctly

### Set up the measurement info
exp = select_experiment('V0_0113A', 'dev48') # select experiment (info for the database)

# need to make sure the sweep is correctly described in the beginning!
meas = MeasurementExt(station, exp=exp, 
                      hard_sweep_detector=detector,
                      soft_sweep_params=[
                          yoko.voltage,
                          ivvi.dac7
#                           ro_attenuator.attenuation,
                      ],
                     )

#meas.register_parameter(ivvi.dac7)
# and finally, the measurement loop
meas.write_period = 2 # how often to write to the DB [s]
with meas.run() as datasaver:
    
    # this is to connect the database to the plottr.
    plot_subscriber = QcodesDatasetSubscriber(datasaver.dataset)
    datasaver.dataset.subscribe(plot_subscriber, state=[])
    
    # the loop itself is trivial
#     for att in ro_atts:
#         ro_attenuator.attenuation(att)
    for v in biasvals:
        yoko.voltage(v)
        for p in plungervals:
            ivvi.dac7(p)
        

            detector.acquisition() # this gets the data from the alazar
            datasaver.add_result(*meas.get_result()) # this adds all data to the datasaver.

No ro_gate defined in channel map. No readout gate pulse will be generated.
No src_gate defined in channel map. No src gate pulse will be generated.


Instrument sequence already exists.
Instrument detector already exists.
Waveforms done
['wfm_1_1_1']
True debug
[['wfm_1_1_1']]
Starting experimental run with id: 568


In [59]:
ami.field(0)

### Hards sweep

In [23]:
fg.ch1.state('OFF')

In [257]:
# set IQ acquisition controller
acq = avgiq_acq
acq.trigger_func = awg_trigger_func

sequence = create_inst(AWGGateRamp1D, 'awg_ramp', awg=awg, force_new_instance=True)
station = add2station(station, sequence)
detector = create_inst(AlazarDetector, name='detector', acqctl=acq, force_new_instance=True)
station = add2station(station, detector)

sequence.sample_rate(1e7)
# sequence.step_len(55e-6)
sequence.step_len(100e-6)
sequence.start_voltage(-0.04)
sequence.stop_voltage(0.04)
sequence.voltage_points(401)
sequence.setup(start_awg=True)

# configure the Alazar detector
detector.set_sweeper(sequence) # set the sequence (it's just one dummy sweep-point here)
detector.demod_frq(hetsrc.IF()) # demod frequency from the sources
detector.acq_time(sequence.step_len() - 5e-6) # not too long integration time ber buffer
detector.configure_alazar(
    records_per_buffer=sequence.voltage_points(),
    buffers_per_acquisition=2, # set averaging by choosing the number of buffers
)
detector.setup() # this makes sure all params are set correctly

print('frequency ' + str(1/(sequence.step_len()*sequence.voltage_points())) + " Hz")

Instrument awg_ramp already exists.
Instrument detector already exists.
Waveforms done
['wfm_1_1_1']
True debug
[['wfm_1_1_1']]


ValueError: ('-1 is invalid: must be between -0.3 and 1.55 inclusive; Parameter: awg_ch1.marker1_low', 'setting awg_ch1_marker1_low to -1')

In [ ]:
points=401
step_len=105e-6
print(step_len*points)
print(1/(step_len*points))

In [54]:
hetsrc.frequency(230e6)
hetsrc.IF(10e6)
hetsrc.RF.on()
hetsrc.LO.on()
hetsrc.RF.power(-40)
hetsrc.LO.power(15)

In [50]:
SAMPLE = 'device48'

In [51]:
ivvi.set_dacs_zero()

In [55]:
ivvi_setup.dac3(-3)
exp = select_experiment('V0_0113A', SAMPLE) # select experiment (info for the database)

# need to make sure the sweep is correctly described in the beginning!
meas = MeasurementExt(station, exp=exp, 
                      hard_sweep_detector=detector,
                      soft_sweep_params=[
                          #hetsrc.frequency,
                          ivvi.dac3
#                           ro_attenuator.attenuation,
                      ],
                     )

#meas.register_parameter(ivvi.dac7)
# and finally, the measurement loop
meas.write_period = 2 # how often to write to the DB [s]
with meas.run() as datasaver:
    
    # this is to connect the database to the plottr.
    plot_subscriber = QcodesDatasetSubscriber(datasaver.dataset)
    datasaver.dataset.subscribe(plot_subscriber, state=[])
    
    # the loop itself is trivial
#     for att in ro_atts:
#         ro_attenuator.attenuation(att)
    for BG in np.linspace(-3,-4,401):
        ivvi_setup.dac3(BG)
        detector.acquisition() # this gets the data from the alazar
        datasaver.add_result(*meas.get_result()) # this adds all data to the datasaver.

Starting experimental run with id: 530


### Frequency generator activated sweep

In [20]:
def awg_trigger_func(do_start):
    if do_start:
        #fg.ch1.state('ON')
        #station.awg.play()
        fg.ch1.burst.trigger()
#     else:
        #fg.ch1.state('OFF')
        #station.awg.stop()
#         fg.ch1.burst.trigger()
        
for ctl in [raw_acq, avgdemod_acq, avgiq_acq]:
    ctl.trigger_func = awg_trigger_func

In [21]:
AWGGateRamp1D.sweep_wait = 'first'

In [45]:
hetsrc.frequency(230e6)
hetsrc.IF(1e6)
hetsrc.RF.on()
hetsrc.LO.on()
hetsrc.RF.power(-40)
hetsrc.LO.power(10)

# set IQ acquisition controller
acq = avgiq_acq
acq.trigger_func = awg_trigger_func

sequence = create_inst(AWGGateRamp1D, 'awg_ramp', awg=awg, force_new_instance=True)
station = add2station(station, sequence)
detector = create_inst(AlazarDetector, name='detector', acqctl=acq, force_new_instance=True)
station = add2station(station, detector)

sequence.alazar_trigger_len = 1e-6
sequence.sample_rate(1e9)
sequence.initial_delay(128e-6)
sequence.step_len(1024e-6)
sequence.start_voltage(-0.025)
sequence.stop_voltage(0.025)
sequence.voltage_points(10)
sequence.ramp_down_time(128e-6)
sequence.setup(start_awg=True)

# configure the Alazar detector
detector.set_sweeper(sequence) # set the sequence (it's just one dummy sweep-point here)
detector.demod_frq(hetsrc.IF()) # demod frequency from the sources
detector.acq_time(sequence.step_len() - 5e-6) # not too long integration time ber buffer
detector.configure_alazar(
    records_per_buffer=sequence.voltage_points(),
    buffers_per_acquisition=1, # set averaging by choosing the number of buffers
)
detector.setup() # this makes sure all params are set correctly

Instrument awg_ramp already exists.
Instrument detector already exists.
Waveforms done
['wfm_1_1_1']
True debug
[['wfm_1_1_1']]


In [239]:
acq.samples_per_record() / 0.2e9

1.536e-05

In [28]:
yoko.on()

In [46]:
print('frequency ' + str(1/(sequence.step_len()*sequence.voltage_points())) + " Hz")

ivvi_setup.dac3(-6.5)
ivvi.dac6(0)
ivvi.dac7(-1200)
cuttervals = np.linspace(0, 400, 101)
plungervals = np.linspace(-500, 600, 401)

exp = select_experiment('V0_0113A', 'dev48') # select experiment (info for the database)

# need to make sure the sweep is correctly described in the beginning!
meas = MeasurementExt(station, exp=exp, 
                      hard_sweep_detector=detector,
                      soft_sweep_params=[
                          #hetsrc.frequency,
                          #ivvi.dac6,
                          ivvi.dac7,
                          #ami.field,
#                           ro_attenuator.attenuation,
                      ],
                     )

#meas.register_parameter(ivvi.dac7)
# and finally, the measurement loop
meas.write_period = 2 # how often to write to the DB [s]
with meas.run() as datasaver:
    
    # this is to connect the database to the plottr.
    plot_subscriber = QcodesDatasetSubscriber(datasaver.dataset)
    datasaver.dataset.subscribe(plot_subscriber, state=[])
    
    # the loop itself is trivial
#     for att in ro_atts:
#         ro_attenuator.attenuation(att)
    #for f in [1, 1.5]:
        #ami.spherical([field,105,90])
#     for c in cuttervals:
#         ivvi.dac6(c)
    for p in plungervals:
        ivvi.dac7(p)
#         fg.ch1.state('ON')
#         time.sleep(0.01)
        detector.acquisition() # this gets the data from the alazar
#         fg.ch1.state('OFF')
        datasaver.add_result(*meas.get_result()) # this adds all data to the datasaver.

frequency 97.65625000000001 Hz
Starting experimental run with id: 591


RuntimeError: ('error 579: ApiWaitTimeout: operation did not finish during timeout interval. Check your trigger. from function AlazarWaitAsyncBufferComplete with args: [2532, c_void_p(2671479554048), 10000]', 'getting avgiq_acq_acquisition', 'getting detector_acquisition')

Connected to: None awg (serial:None, firmware:None) in 484.15s


In [36]:
fg.ch1.burst.trigger()

In [156]:
fg.ch1.burst.source('MAN')

In [157]:
fg.ch1.burst.on('ON')

In [39]:
for i in range(50):
    fg.ch1.burst.trigger()
    time.sleep(1)

In [127]:
fg.ch1.freq(500)

In [362]:
data = acq.acquisition()

RuntimeError: ('error 579: ApiWaitTimeout: operation did not finish during timeout interval. Check your trigger. from function AlazarWaitAsyncBufferComplete with args: [2584, c_void_p(2903496196096), 10000]', 'getting avgiq_acq_acquisition')

In [205]:
data.shape

(401, 2)

In [286]:
for i in range(10):
    data = acq.acquisition()
    time.sleep(0.1)

RuntimeError: ('error 579: ApiWaitTimeout: operation did not finish during timeout interval. Check your trigger. from function AlazarWaitAsyncBufferComplete with args: [2584, c_void_p(2903496196096), 10000]', 'getting avgiq_acq_acquisition')